Welcome new ADAM user!

   This notebook will guide you through a few ADAM features and validate that your system is configured correctly.  

   Before starting this notebook, you should have already run the config_demo.ipynb notebook on your system (located in adam_home/demos).  Additionally, please ensure that you have installed Astroquery on your system (instructions in FAQ).    
   
   
Currently, as of 07-MAR-19, this notebook supports the following features. 

-Pulls trajectory data for a given object and timeframe from NASA JPL

-Propagates trajectory for that same object in ADAM. 

-Plots orbits for ADAM and JPL data.  (ADAM orbit is currently plotted in ecliptic plane)

-Calculates and displays RIC for ADAM vs JPL

-Exports ADAM Data in STK format

-Exports ADAM Data in CSV format



In Development

-Transform ADAM data to rotate from ecliptic to celestial plane.  


NOTES: 

-This version currently defaults to the dev server since, as of 07-MAR-19, the production side is pending updates.

-This version is written to read the astro folder from within "adam_home".  You may need to point the code in the next cell to your Astro folder.    
 


# This imports libraries from ADAM and Python

In [ ]:
# This specifies the path to the adam_home folder and configuration file on your system.
import sys
from os.path import expanduser
adam_home_defined = expanduser("~") + "/adam_home" # default location
config_folder=expanduser("~") + "/adam_home/config" # default location  # Works,  19 Feb -LRH
import adam

# This imports the necessary libraries from ADAM and JPL as well as other packages. 
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
import time
import os
from adam import Service
from adam import Batch
from adam import PropagationParams
from adam import OpmParams
from adam import BatchRunManager
from adam import ConfigManager
import unittest
import datetime
import os
import numpy as np
import numpy.testing as npt
from adam.batch import StateSummary
from adam.batch import PropagationResults

#Astro Query
from astroquery.jplhorizons import Horizons
from astroquery.jplhorizons import conf
conf.horizons_server = 'https://ssd.jpl.nasa.gov/horizons_batch.cgi'



#NOTE: This version is written to read the astro folder from within "adam_home". You may need to move astro to this folder. 
#If the astro folder is not in adam_home, please modify the next line to point to your "astro" folder. 
#sys.path.insert(0,'C:\\Users\\Lowell\\adam_home\\astro') 
#astro Lib
from astro import Transform
from astro import asteroid_database
  


#Other Lib
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 



# Identify Asteroid and pull from JPL

In [ ]:
print("Enter asteroid target number. e.g for Eros use 433. IDs can be searched here: https://ssd.jpl.nasa.gov/sbdb.cgi#top ")
asteroid = input() #in form of target number i.e. for Eros use 433

# You can edit the inputs below to change the start/stop times and the step size.  
start_time_str='2016-10-04T00:00:00' #YYYY-Mon-Dy {HH:MM}   #Default: start_time_str='2016-10-04T00:00:00'
end_time_str='2017-10-11T00:00:00'   # Default: end_time_str='2017-10-11T00:00:00'
step = '1d'   #Default:  1d

In [ ]:
#  Repeated 2 cells down???

#call the object

obj = Horizons(id=str(asteroid), location=None ,epochs={'start':start_time_str, 'stop':end_time_str,'step':step})
vec = obj.vectors() #state vector at each epoch, inital state vector is initial epoch
#vec is the entire epheremris over the specified timeframe, the inital state vector (to be put into ADAM) is vec's first row

# vec  #Vec is commented out to improve readiblity, otherwise it displays a full page of data.  Should we keep it?  
#  This comes up several more times,  maybe we need a way for the user to output these to a different file type?

In [ ]:
#Simplified Dataframe

eph_JPL_df = pd.DataFrame(
    {'Time [Barycentric TBD]': vec['datetime_jd'],
     'x [m]': vec['x']*1.496e+11,
     'y [m]': vec['y']*1.496e+11,
     'z [m]': vec['z']*1.496e+11,
     'x_dot [m/s]': vec['vx']*1731456.84,
     'y_dot [m/s]': vec['vy']*1731456.84,
     'z_dot [m/s]': vec['vz']*1731456.84
     
    })



# eph_JPL_df    # Commented out to improve readibility. 

In [ ]:
#call the object

obj = Horizons(id=str(asteroid), location=None ,epochs={'start':start_time_str, 'stop':end_time_str,'step':step})
vec = obj.vectors() #state vector at each epoch, inital state vector is initial epoch

#JPL Cartesian State Vector + Data Frame
JPL_SV = np.array([vec[i][0] for i in vec.columns])
JPL_SV_df_cart = pd.DataFrame(JPL_SV, index= vec.columns, columns=list(' ')) 
#JPL r and v vectors
JPL_r = np.array([np.float(i) for i in JPL_SV[5:8]])*1.496e+8
JPL_v = np.array([np.float(i) for i in JPL_SV[8:11]])*1731.46
print("Position: %s km" % JPL_r)
print("Velocity: %s km/s" % JPL_v)
#########################################################################
#Converting to Keplerian 
d2r = np.pi/180.
r2d = 180./np.pi
trans = Transform()



AU = 149597870.0     # conversion from AU to km
JPL_a, JPL_e, JPL_i, JPL_node, JPL_w, JPL_TA = trans.cartesian_to_classical(JPL_r,JPL_v)
kep_col = np.array(['SMA','Ecc','Inc','RAAN','Arg of per','True anomaly'])


JPL_SV_df_kep = pd.DataFrame(np.array([JPL_a, JPL_e, JPL_i*r2d, JPL_node*r2d, JPL_w*r2d, JPL_TA*r2d]), index= kep_col, columns=list(' ')) 

#Combining two dataframes
JPL_SV_df = pd.concat([JPL_SV_df_cart, JPL_SV_df_kep], axis=0) 
# JPL_SV_df  # Again, commented out to avoid excessive outputs. 



In [ ]:
# Import Minor Planet Center Queries

from astroquery.mpc import MPC
mpc = MPC()
result = mpc.query_object_async(number=asteroid, target_type="asteroid")
#result.json()   # Remove # to see results

#[{'absolute_magnitude': '3.34', 'aphelion_distance': '2.976', 'arc_length': 79247, 'argument_of_perihelion': '73.11528', 'ascending_node': '80.3099167', 'critical_list_numbered_object': False, 'delta_v': 10.5, 'designation': None, 'earth_moid': 1.59353, 'eccentricity': '0.0755347', 'epoch': '2018-03-23.0', 'epoch_jd': '2458200.5', 'first_observation_date_used': '1801-01-31.0', 'first_opposition_used': '1801', 'inclination': '10.59351', 'jupiter_moid': 2.09509, 'km_neo': False, 'last_observation_date_used': '2018-01-20.0', 'last_opposition_used': '2018', 'mars_moid': 0.939285, 'mean_anomaly': '352.23052', 'mean_daily_motion': '0.2141308', 'mercury_moid': 2.18454, 'name': 'Ceres', 'neo': False, 'number': 1, 'observations': 6689, 'oppositions': 114, 'orbit_type': 0, 'orbit_uncertainty': '0', 'p_vector_x': '-0.87827466', 'p_vector_y': '0.33795664', 'p_vector_z': '0.33825868', 'perihelion_date': '2018-04-28.28378', 'perihelion_date_jd': '2458236.78378', 'perihelion_distance': '2.5580384', 'period': '4.6', 'pha': False, 'phase_slope': '0.12', 'q_vector_x': '-0.44248615', 'q_vector_y': '-0.84255514', 'q_vector_z': '-0.30709419', 'residual_rms': '0.6', 'saturn_moid': 6.38856, 'semimajor_axis': '2.7670463', 'tisserand_jupiter': 3.3, 'updated_at': '2018-02-26T17:29:46Z', 'uranus_moid': 15.6642, 'venus_moid': 1.84632}]

#MPC Eph database
#https://minorplanetcenter.net/iau/MPEph/MPEph.html?format=json

In [ ]:
#result.content   # Probably don't need to keep this one.  

# Propagate through ADAM

In [ ]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
config = ConfigManager(config_folder + '/adam_config.json').get_config("dev")  # remove "dev" to default to prod
service = Service(config)
service.setup()

auth_rest = AuthenticatingRestProxy(RestRequests(config.get_url()), config.get_token())

In [ ]:
working_project = service.new_working_project() # cleans up old batches

In [ ]:
#Set Keplerian Parameters

start_time_str_Z=start_time_str+'Z'
end_time_str_Z=end_time_str+'Z'


#keplerian Method:

keplerian_elements = {
    'semi_major_axis_km': np.float(JPL_SV_df.iat[14,0]),
    'eccentricity': np.float(JPL_SV_df.iat[15,0]),
    'inclination_deg': np.float(JPL_SV_df.iat[16,0]),
    'ra_of_asc_node_deg': np.float(JPL_SV_df.iat[17,0]),
    'arg_of_pericenter_deg': np.float(JPL_SV_df.iat[18,0]),
    'true_anomaly_deg': np.float(JPL_SV_df.iat[19,0]),
    'gm': 1.327124400419394E11                                    
}

propagation_params = PropagationParams({
    'start_time': start_time_str_Z,
    'end_time': end_time_str_Z,
    'project_uuid': config.get_workspace(),
    'description': 'Created by test at ' + start_time_str
})
opm_params_templ = {
    'epoch': start_time_str_Z,
    'object_name': (JPL_SV_df.iat[0,0]),    # object ID
    'object_id':asteroid
    #'comment':"Keplerian Coord System"                      how do I change OPM stuff
    
    # state_vector or keplerian_elements will be set later.

    #'mass': 500.5,
    #'solar_rad_area': 25.2,
    #'solar_rad_coeff': 1.2,
    #'drag_area': 33.3,
    #'drag_coeff': 2.5
}
###################################################################################
#Create keplerian batch files to be put into ADAM
def make_keplerian_batches(start_time_str, end_time_st):
    keplerian_opm_params = opm_params_templ.copy()
    keplerian_opm_params['keplerian_elements'] = keplerian_elements

    keplerian = Batch(propagation_params, OpmParams(keplerian_opm_params))
    return  keplerian


kep_batch= make_keplerian_batches(start_time_str, end_time_str)
print(kep_batch.get_opm_params().generate_opm())

# Submit and wait until batch run is ready
batches_module = Batches(auth_rest)
BatchRunManager(batches_module, [kep_batch]).run()

###################################################################################
# Retrieve the vector at the end of the state
def kep_single_run(self, start_time_str, end_time_str):
    
    #start_time_str = start.isoformat() + 'Z'
    #end_time_str = end.isoformat() + 'Z'

    keplerian = make_keplerian_batches(start_time_str, end_time_str) #batch objets
    print(keplerian.get_state_summary().get_parts_count())

    BatchRunManager(self.service.get_batches_module(), [kep_batch]).run()

    keplerian_end_state = kep_batch.get_results().get_end_state_vector()
    return keplerian_end_state

# Get the end state vector (uncomment to use)
end_state_vector = kep_batch.get_results().get_end_state_vector()
print("State vector at the end of propagation:")
print(end_state_vector)
###################################################################################
#Get status and parts count
parts_count = kep_batch.get_state_summary().get_parts_count()
print("Final state: %s, part count %s\n" % (kep_batch.get_calc_state(), parts_count))


In [ ]:
# Get ephemeris of specified part
part_to_get = 0
eph = kep_batch.get_results().get_parts()[part_to_get].get_ephemeris()
#print("Ephemeris:")  #Commented out to improve readability.  #Moved to end of notebook
#print(eph)

################################################################################
#Parse and Clean Above String
_ ,eph_temp = eph.split("EphemerisTimePosVel\n\n", 1)
eph_temp, _ = eph_temp.split("\n\nEND Ephemeris", 1)
eph_temp

eph_temp_split=eph_temp.split(' ')
#eph_temp_split

eph_split_fix=[]
atomic_times=[]
atomic_times.append(np.float(eph_temp_split[0]))
for i in range(1, len(eph_temp_split[0::6])-1):
    #calculate atomic time
    temp = eph_temp_split[0::6]
    temp_2=temp[i]
    #print(temp_2)
    eph_split_again,atomic_time= temp_2.split('\n')
    atomic_times.append(np.float(atomic_time))
    eph_split_fix.append(np.float(eph_split_again))
eph_split_fix.append(np.float(eph_temp_split[-1]))

#print("Atomic Times: ", atomic_times)  #Commented out to improve readability.

eph_split_fix.insert(0,0)
#eph_split_fix.append(0)

eph_temp_split[0::6]=eph_split_fix

for i in range(0,len(eph_temp_split)):
    eph_temp_split[i]=np.float(eph_temp_split[i])

eph_Adam=eph_temp_split[1:len(eph_temp_split)]

#change from long list to list of 6 elements of statevector
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

eph_Adam_vectors=[]
for group in chunker(eph_Adam, 6):
    eph_Adam_vectors.append((group))
    
eph_Adam_x=[]
eph_Adam_y=[]
eph_Adam_z=[]
eph_Adam_xdot=[]
eph_Adam_ydot=[]
eph_Adam_zdot=[]
for i in range(0,len(eph_Adam_vectors)):
    temp=eph_Adam_vectors[i]
    eph_Adam_x.append(temp[0])
    eph_Adam_y.append(temp[1])
    eph_Adam_z.append(temp[2])
    eph_Adam_xdot.append(temp[3])
    eph_Adam_ydot.append(temp[4])
    eph_Adam_zdot.append(temp[5])
#############################################################################
#Construct ADAM dataframe for the JPL statevector
eph_JPLtoAdam_df = pd.DataFrame(
    {'Time [Atomic UTC]': atomic_times,
     'x [m]': eph_Adam_x,
     'y [m]': eph_Adam_y,
     'z [m]': eph_Adam_z,
     'x_dot [m/s]': eph_Adam_xdot,
     'y_dot [m/s]': eph_Adam_ydot,
     'z_dot [m/s]': eph_Adam_zdot
     
    })

#eph_JPLtoAdam_df  #Commented out to improve readability.


In [ ]:
#(eph_JPL_df)  #Commented out to improve readability.

In [ ]:
#(eph_JPL_df)-eph_JPLtoAdam_df #Commented out to improve readability.

# Plot Orbits for JPL and ADAM.

In [ ]:
# Visualize ADAM and JPL orbits for the object.

# Let's just do Earth and object (and Sun) for now.  
earth = Horizons(id='Geocenter', location=None, epochs={'start':start_time_str, 'stop':end_time_str, 'step':'1d'}, id_type = 'majorbody')
vEarth = earth.vectors()
#sun = Horizons(id='Sun', location=None, epochs={'start':start_time_str, 'stop':end_time_str, 'step':'1d'}, id_type = 'majorbody')
#vSun = sun.vectors()

AU = 1.49597870e+11  # Astronomical Units in meters  

#  Get the points for JPL object   #Works
n = len(vec)
xObj = [1]*n               # initialize X position array
yObj = [1]*n               # initialize Y position array
zObj = [1]*n               # initialize Z position array

for i in range(0,n):             
            xObj[i] = vec['x'][i]
            yObj[i] = vec['y'][i]
            zObj[i] = vec['z'][i]
            
            
            
#  Get the points for ADAM object             # Doesn't work
n = len(eph_Adam_vectors)           
xADAM_Obj = [1]*n               # initialize X position array
yADAM_Obj = [1]*n               # initialize Y position array
zADAM_Obj = [1]*n               # initialize Z position array

for i in range(0,n):             
            #xADAM_Obj[i] = eph_Adam_vectors['x'][i]  # Not working
            xADAM_Obj[i] = (eph_Adam_x[i]/AU) #FIXED    21FEB19,   converted AU to meters   # There's probably a more sophisticated way to do this    # Not working
            yADAM_Obj[i] = (eph_Adam_y[i]/AU) 
            yADAM_Obj[i] = (eph_Adam_z[i]/AU) 
            
            

            
# Earth points  # Works
            
n = len(vEarth)
xEarth = [1]*n               # initialize X position array
yEarth = [1]*n               # initialize Y position array
zEarth = [1]*n               # initialize Z position array

for i in range(0,n):             
            xEarth[i] = vEarth['x'][i]
            yEarth[i] = vEarth['y'][i]
            zEarth[i] = vEarth['z'][i]
            



# (In Development) Transform ADAM Vectors from ICRF to EMEME2000

In [ ]:


#ADAM_ICRF = Transform.EMEME2000_to_ICRF(xADAM_Obj,yADAM_Obj,zADAM_Obj)

# This is shifting it down,  rahter than rotating it. 
# Try plane rotation?

            
#n = len(eph_Adam_vectors) #len(ADAM_ICRF)    # I had to use Eph_ADAM because ADAM_ICRF has length = 3   ...
#TxvUdo = [1]*n               # initialize X position array
#TyvUdo = [1]*n               # initialize Y position array
#TzvUdo = [1]*n               # initialize Z position array

#for i in range(0,n):             
#            TxvUdo[i] = ADAM_ICRF[0][i]
#            TyvUdo[i] = ADAM_ICRF[1][i]
#            TzvUdo[i] = ADAM_ICRF[2][i]
            
    


In [ ]:
#  Plot ADAM and JPL 
  
plot_size = max([np.abs(xObj).max() ]) 

size = [-plot_size+0.1, plot_size+.1]  


fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection ='3d')   # Not working 08FEB19  # Resolved with import in 1st cell

# Plot JPL Object
ax.plot(xObj,yObj,zObj, c = 'orange')

#Plot ADAM Object
ax.plot(xADAM_Obj,yADAM_Obj,zADAM_Obj,c='red')  #FIXED    21FEB19,   converted AU to KM  # Not working
#ax.plot(eph_Adam_x,eph_Adam_y,eph_Adam_z,c='red')  # Not working


#Plot ADAM Trans-to_ICRF Object
# ax.plot(TxvUdo,TyvUdo,TzvUdo,c='purple')   # In Development

#Plot Earth
ax.plot(xEarth,yEarth,zEarth,c='blue')



#ax.text(0,1.5,.5, "2002 AJ 129",color='purple')
#ax.text(0,1.5,.2, "1566 Icarus",color='orange')

ax.text(0,1.5,.8, asteroid + " JPL",color='orange')
ax.text(0,1.5,1.3, "Earth",color='blue')
ax.text(0,1.5,1.8, asteroid + " ADAM",color='red')
#ax.text(0,1.5,2.3, asteroid + " ADAM in ICRF",color='purple')   # In development

ax.set_xlim(size), ax.set_ylim(size), ax.set_zlim(size)

plt.show()

# Issue,  Angle is off by 23.5 degrees,  need to convert EMEME2000 to ICRF  

# The following cells allow you ro export the ADAM data to .csv and STK formats.

In [ ]:
#Export to CSV   # Need to fix

print("This will default export location to your adam_home folder, you can modify this in the cell") 

CSV_name = input("Enter file name")
CSV_export  = adam_home_defined + "/" +CSV_name+'.csv'

# Modify following line to change export location
#CSV_export = "C:\Users\Lowell\Documents\Python Scripts\test.csv"  #Enter your path here.  #e.g  C:\Users\Lowell\Documents\Python Scripts\test.csv

#Export ADM Eph
#pd.DataFrame(eph_Adam).to_csv(CSV_export)

pd.DataFrame(eph_Adam_vectors).to_csv(CSV_export)  # Add Adam file  #Works 02MAR19  #Still need to add ADAM File



# Output Ephemeris to STK

In [ ]:
print("Ephemeris:")
print(eph)

# Plot RIC for JPL vs ADAM

In [ ]:
JPL_R = np.array([np.array(eph_JPL_df.iloc[i,4:7])/np.linalg.norm(eph_JPL_df.iloc[i,4:7]) for i in range(0,len(eph_JPL_df))])

JPLtoADAM_R = np.array([np.array(eph_JPLtoAdam_df.iloc[i,1:4])/np.linalg.norm(eph_JPLtoAdam_df.iloc[i,1:4]) for i in range(0,len(eph_JPLtoAdam_df))])

JPL_ADAM_RIC_R = np.array([(JPLtoADAM_R[i]-JPL_R[i])/np.linalg.norm(JPLtoADAM_R[i]-JPL_R[i]) for i in range(0,len(JPL_R))])

print(JPL_ADAM_RIC_R)

JPL_V = np.array([np.array(eph_JPL_df.iloc[i,4:7])/np.sqrt(np.sum(np.square(eph_JPL_df.iloc[i,4:7]))) for i in range(0,len(eph_JPL_df))])
JPLtoADAM_V = np.array([np.array(eph_JPLtoAdam_df.iloc[i,4:7])/np.sqrt(np.sum(np.square(eph_JPLtoAdam_df.iloc[i,4:7]))) for i in range(0,len(eph_JPLtoAdam_df))])


JPL_ADAM_RIC_V = np.array([(JPLtoADAM_V[i]-JPL_V[i])/np.linalg.norm(JPLtoADAM_V[i]-JPL_V[i]) for i in range(0,len(JPL_R))])
print(JPL_ADAM_RIC_V)

In [ ]:
JPL_ADAM_RIC_C = np.array([np.cross(JPL_ADAM_RIC_R[i],JPL_ADAM_RIC_V[i]/np.linalg.norm(np.cross(JPL_ADAM_RIC_R[i],JPL_ADAM_RIC_V[i]))) for i in range(0,len(JPL_R))])

JPL_ADAM_RIC_C

In [ ]:
JPL_ADAM_RIC_I = np.array([np.cross(JPL_ADAM_RIC_R[i],JPL_ADAM_RIC_C[i]/np.linalg.norm(np.cross(JPL_ADAM_RIC_R[i],JPL_ADAM_RIC_C[i]))) for i in range(0,len(JPL_R))])

JPL_ADAM_RIC_I

In [ ]:

atomic_times=atomic_times[0:len(atomic_times)-1]

plt.figure(figsize=(10,5))
plt.grid(True,ls='--')
plt.plot(atomic_times,JPL_ADAM_RIC_R[:,0],linewidth=3,label="Radial")
plt.plot(atomic_times,JPL_ADAM_RIC_I[:,0],linewidth=3,label="Cross Track")
plt.plot(atomic_times,JPL_ADAM_RIC_C[:,0],linewidth=3,label="In Track")
plt.title("RIC for JPL vs ADAM (along x)")
plt.legend()
plt.xlabel('Atomic Time')

plt.figure(figsize=(10,5))
plt.plot(atomic_times,JPL_ADAM_RIC_R[:,1],linewidth=3,label="Radial")
plt.plot(atomic_times,JPL_ADAM_RIC_I[:,1],linewidth=3,label="Cross Track")
plt.plot(atomic_times,JPL_ADAM_RIC_C[:,1],linewidth=3,label="In Track")
plt.grid(True)
plt.title("RIC for JPL vs ADAM (along y)")
plt.xlabel('Atomic Time')
plt.legend()

plt.figure(figsize=(10,5))
plt.plot(atomic_times,JPL_ADAM_RIC_R[:,2],linewidth=3,label="Radial")
plt.plot(atomic_times,JPL_ADAM_RIC_I[:,2],linewidth=3,label="Cross Track")
plt.plot(atomic_times,JPL_ADAM_RIC_C[:,2],linewidth=3,label="In Track")
plt.title("RIC for JPL vs ADAM (along z)")
plt.grid(True)
plt.xlabel('Atomic Time')
plt.legend()



In [ ]:
len(atomic_times[0:len(atomic_times)-1])

In [ ]:
plt.show()

Frequently Asked Questions (FAQs)

How to install Astroquery?  
   To install Astroquery, pip install --pre astroquery     
More details can be found at astroquery.jplhorizons https://astroquery.readthedocs.io/en/latest/)
